### Thermal Compilance shape

##### Using the required packages

In [1]:
using GridapTopOpt, Gridap

##### Finite Element Parameters

In [1]:
order = 1                               # Finite element order
xmax, ymax = (1.0, 1.0)                 # Domain Size
dom = (0, xmax, 0, ymax)                # Bounding Domain
el_size = (200, 200)                    # Mesh Partition Size
prop_Γ_N = 0.2                          # Γ_N size parameter
prop_Γ_D = 0.2                          # Γ_D size parameter
# Γ_N Neumann Boundary Conditions
f_Γ_N(x) = (x[1] ≈ xmax && ymax/2-ymax*prop_Γ_N/2 - eps() <= x[2] <= ymax/2+ymax*prop_Γ_N/2 + eps())
# Γ_D Drichlet Boundary Conditions
f_Γ_D(x) = (x[1] ≈ 0.0 && (x[2] <= ymax*prop_Γ_D + eps() || x[2] >= ymax-ymax*prop_Γ_D/2 - eps()))

f_Γ_D (generic function with 1 method)

##### Finite Difference Parameters

In [2]:
γ = 0.1                                             # Hamilton-Jaccobi equation time step coefficient
γ_reinit = 0.5                                      # Reininitialization eqn time step coeff
max_steps = floor(Int, order*minimum(el_size)/10)   # Max steps for advection
tol = 1/ (5order^2) /minimum(el_size)               # Reininitialization tolerance

0.001

##### Problem Parameters

In [5]:
k = 1                                           # Diffusivity
g = 1                                           # Heat flow in 
vf = 0.4                                        # Volume fraction constraints
lsf_func = initial_lsf(4, 0.2)                  # Initial level set function
iter_mod = 10                                   # VTK Output module
path = "./Result"
mkpath(path)

"./Result"

##### Model

In [41]:
model = CartesianDiscreteModel(dom, el_size);
update_labels!(1, model, f_Γ_D, "Gamma_D")
update_labels!(2, model, f_Γ_N, "Gamma_N")

##### Triangulations and Measures

In [42]:
Ω = Triangulation(model)
Γ_N = BoundaryTriangulation(model, tags="Gamma_N")
dΩ = Measure(Ω, 2*order)
dΓ_N = Measure(Γ_N, 2*order)

GenericMeasure()

##### Space

In [43]:
reffe = ReferenceFE(lagrangian,VectorValue{2,Float64},order)
reffe_scalar = ReferenceFE(lagrangian,Float64,order)
V = TestFESpace(model,reffe;dirichlet_tags=["Gamma_D"])
U = TrialFESpace(V,VectorValue(0.0,0.0))
V_φ = TestFESpace(model,reffe_scalar)
V_reg = TestFESpace(model,reffe_scalar;dirichlet_tags=["Gamma_N"])
U_reg = TrialFESpace(V_reg,0)

TrialFESpace()

##### Level set and interpolator

In [44]:
φh = interpolate(lsf_func,V_φ)
interp = SmoothErsatzMaterialInterpolation(η = 2*maximum(get_el_Δ(model)))    # η = 2 ×  maximum side length of an element.
I,H,DH,ρ = interp.I,interp.H,interp.DH,interp.ρ

(GridapTopOpt.var"#I#298"{GridapTopOpt.var"#H#296"{Vector{Float64}}, Vector{Float64}}(GridapTopOpt.var"#H#296"{Vector{Float64}}([0.01]), [0.001]), GridapTopOpt.var"#H#296"{Vector{Float64}}([0.01]), GridapTopOpt.var"#DH#297"{Vector{Float64}}([0.01]), GridapTopOpt.var"#ρ#299"{GridapTopOpt.var"#H#296"{Vector{Float64}}}(GridapTopOpt.var"#H#296"{Vector{Float64}}([0.01])))

Weak formulation

In [45]:
a(u,v,φ) = ∫((I ∘ φ)*(C ⊙ ε(u) ⊙ ε(v)))dΩ
l(v,φ) = ∫(v·g)dΓ_N
state_map = AffineFEStateMap(a,l,U,V,V_φ)#,U_reg,φh,dΩ,dΓ_N)

AffineFEStateMap

In [46]:
# Objective and constraints
    J(u,φ) = ∫((I ∘ φ)*(C ⊙ ε(u) ⊙ ε(u)))dΩ
    dJ(q,u,φ) = ∫((C ⊙ ε(u) ⊙ ε(u))*q*(DH ∘ φ)*(norm ∘ ∇(φ)))dΩ
    vol_D = sum(∫(1)dΩ)
    C1(u,φ) = ∫(((ρ ∘ φ) - vf)/vol_D)dΩ
    dC1(q,u,φ) = ∫(-1/vol_D*q*(DH ∘ φ)*(norm ∘ ∇(φ)))dΩ
    pcfs = PDEConstrainedFunctionals(J,[C1],state_map,analytic_dJ=dJ,analytic_dC=[dC1])

PDEConstrainedFunctionals:
    num_constraints: 1

In [47]:
# Velocity extension
α = 4max_steps*γ*maximum(get_el_Δ(model))
a_hilb(p,q) = ∫(α^2*∇(p)⋅∇(q) + p*q)dΩ;
vel_ext = VelocityExtension(a_hilb,U_reg,V_reg)
evo = FiniteDifferenceEvolver(FirstOrderStencil(2,Float64),model,V_φ;max_steps)
reinit = FiniteDifferenceReinitialiser(FirstOrderStencil(2,Float64),model,V_φ;tol,γ_reinit)
ls_evo = LevelSetEvolution(evo,reinit)

LevelSetEvolution{FiniteDifferenceEvolver{1}, FiniteDifferenceReinitialiser{1}}(FiniteDifferenceEvolver{1}(FirstOrderStencil{2, Float64}(), CartesianDiscreteModel(), UnconstrainedFESpace(), [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  40392, 40393, 40394, 40395, 40396, 40397, 40398, 40399, 40400, 40401], (isperiodic = (false, false), Δ = (0.005, 0.005), ndof = (201, 201), max_steps = 20, correct_ls = false)), FiniteDifferenceReinitialiser{1}(FirstOrderStencil{2, Float64}(), CartesianDiscreteModel(), UnconstrainedFESpace(), [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  40392, 40393, 40394, 40395, 40396, 40397, 40398, 40399, 40400, 40401], (isperiodic = (false, false), Δ = (0.005, 0.005), ndof = (201, 201), max_steps = 2000, tol = 0.001, γ_reinit = 0.5, correct_ls = false)))

In [48]:
# Optimiser
optimiser = AugmentedLagrangian(pcfs,ls_evo,vel_ext,φh;γ,verbose=true,constraint_names=[:Vol])

AugmentedLagrangian

In [49]:
# Solve
    for (it,uh,φh) in optimiser
        data = ["φ"=>φh,"H(φ)"=>(H ∘ φh),"|∇(φ)|"=>(norm ∘ ∇(φh)),"uh"=>uh]
        iszero(it % iter_mod) && writevtk(Ω,path*"out$it",cellfields=data)
        write_history(path*"/history_$max_steps,$tol.txt",get_history(optimiser))
    end 

UndefVarError: UndefVarError: `C` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [50]:
# Final structure
    it = get_history(optimiser).niter; uh = get_state(pcfs)
    writevtk(Ω,path*"out$it,$max_steps,$tol",cellfields=["φ"=>φh,"H(φ)"=>(H ∘ φh),"|∇(φ)|"=>(norm ∘ ∇(φh)),"uh"=>uh])

AssertionError: AssertionError:   You must build the cache before using get_state. This can be achieved by either
  solving your problem with my_state_map(φh) or by running build_cache!(my_state_map,φh)
